In [4]:
import pandas as pd

In [5]:
fname = 'flickr_logos_dataset/flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt'

In [6]:
# Read the text file into a pandas DataFrame
columns = ["imgname", "classname", "class", "xmin", "ymin", "xmax", "ymax"]
data_df = pd.read_csv(fname, sep=' ', header=None, names=columns,index_col=False)

In [7]:
data_df

,imgname,classname,class,xmin,ymin,xmax,ymax
0,144503924.jpg,Adidas,1,38,12,234,142
1,2451569770.jpg,Adidas,1,242,208,413,331
2,390321909.jpg,Adidas,1,13,5,89,60
3,4761260517.jpg,Adidas,1,43,122,358,354
4,4763210295.jpg,Adidas,1,83,63,130,93
...,...,...,...,...,...,...,...
4531,2126991906.jpg,Yahoo,6,15,6,253,54
4532,217288720.jpg,Yahoo,6,136,161,304,222
4533,2472817996.jpg,Yahoo,6,2,4,499,106
4534,2514220918.jpg,Yahoo,6,1,69,342,157


In [8]:
data_df['class'].max()

6

In [9]:
data_df['classname'].unique()

array(['Adidas', 'Apple', 'BMW', 'Citroen', 'Cocacola', 'DHL', 'Fedex',
       'Ferrari', 'Ford', 'Google', 'Heineken', 'HP', 'Intel',
       'McDonalds', 'Mini', 'Nbc', 'Nike', 'Pepsi', 'Porsche', 'Puma',
       'RedBull', 'Sprite', 'Starbucks', 'Texaco', 'Unicef', 'Vodafone',
       'Yahoo'], dtype=object)

In [10]:
class_names = data_df['classname'].unique()        

In [11]:
class_name_to_label = {class_name: label for label, class_name in enumerate(class_names)}

In [12]:
class_name_to_label

{'Adidas': 0,
 'Apple': 1,
 'BMW': 2,
 'Citroen': 3,
 'Cocacola': 4,
 'DHL': 5,
 'Fedex': 6,
 'Ferrari': 7,
 'Ford': 8,
 'Google': 9,
 'Heineken': 10,
 'HP': 11,
 'Intel': 12,
 'McDonalds': 13,
 'Mini': 14,
 'Nbc': 15,
 'Nike': 16,
 'Pepsi': 17,
 'Porsche': 18,
 'Puma': 19,
 'RedBull': 20,
 'Sprite': 21,
 'Starbucks': 22,
 'Texaco': 23,
 'Unicef': 24,
 'Vodafone': 25,
 'Yahoo': 26}